## First Step: Data Extraction and Preperation

### Get data from the arXiv database:

In [1]:
!pip install feedparser

import urllib.request
import feedparser

# Base api query url
base_url = 'http://export.arxiv.org/api/query?';

# Search parameters
search_query = 'all:electron' # search for electron in all fields
start = 0                     # retreive the first 5 results
max_results = 10

query = 'search_query=%s&start=%i&max_results=%i' % (search_query,
                                                     start,
                                                     max_results)

# perform a GET request using the base_url and query
response = urllib.request.urlopen(base_url+query).read()

# parse the response using feedparser
feed = feedparser.parse(response)

     |████████████████████████████████| 81kB 6.1MB/s 
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp37-none-any.whl size=6067 sha256=a4727d2ca00c82db2893c61ff0f425ee98f03b452d9661dec08f0d9dce3227cb
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built sgmllib3k


### convert it into a list + dataframe 

In [2]:
#coloums of interasts (maybe?)
col=['title', 'summary', 'authors', 'arxiv_primary_category', 'tags']

# Run through each entry, and fill the information into a list
data_list=[]
for c in col:
  abstract_list=[]
  for entry in feed.entries:
    abstract_list.append(entry.get(c))
  data_list.append(abstract_list)

# convert into a panda dataframe (maybe more visible + have some pros I might need)
import pandas as pd
data_df = pd.DataFrame(data_list,index=col)
data_df=data_df.T


In [3]:
data_df.to_csv('arXiv10.csv') 

In [4]:
data_df

title  ...                                               tags
0  Impact of Electron-Electron Cusp on Configurat...  ...  [{'term': 'cond-mat.str-el', 'scheme': 'http:/...
1  Electron thermal conductivity owing to collisi...  ...  [{'term': 'astro-ph', 'scheme': 'http://arxiv....
2  Electron pairing: from metastable electron pai...  ...  [{'term': 'cond-mat.str-el', 'scheme': 'http:/...
3  Electron Temperature Anisotropy and Electron B...  ...  [{'term': 'physics.space-ph', 'scheme': 'http:...
4  Hamiltonian of a many-electron system with sin...  ...  [{'term': 'cond-mat.supr-con', 'scheme': 'http...
5  Electron-Electron Bremsstrahlung Emission and ...  ...  [{'term': 'astro-ph', 'scheme': 'http://arxiv....
6                  Improved scenario of baryogenesis  ...  [{'term': 'astro-ph', 'scheme': 'http://arxiv....
7  Exact Electron-Pairing Ground States of Tight-...  ...  [{'term': 'cond-mat.str-el', 'scheme': 'http:/...
8  Insights into the Electron-Electron Interactio...  ...  [{'term': 'cond-mat.quant-gas', 'scheme': 'htt...
9  Electron-electron interactions in a weakly scr...  ...  [{'term': 'cond-mat.str-el', 'scheme': 'http:/...

[10 rows x 5 columns]

### tokenizing:

In [5]:
!python -m spacy download en_core_web_lg
import spacy
import en_core_web_lg
nlp = en_core_web_lg.load()

     |████████████████████████████████| 827.9MB 1.1MB/s 
  Created wheel for en-core-web-lg: filename=en_core_web_lg-2.2.5-cp37-none-any.whl size=829180945 sha256=c9b048119799cfff369924e83c9c9f266da3e5ca2b6121d124ac0c3a66bd2411
  Stored in directory: /tmp/pip-ephem-wheel-cache-mc_qspcw/wheels/2a/c1/a6/fc7a877b1efca9bc6a089d6f506f16d3868408f9ff89f8dbfc
Successfully built en-core-web-lg
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


In [6]:
import numpy as np

#taking just the titles (data_list[0]). (maybe to use the summary instaed?)
# using lower case. removing extra spaces and '\n ' 
doc=[nlp.tokenizer(text.lower().replace('\n ','').strip()) for text in data_list[0]]

for example:

In [7]:
II=5;
print([token.rank for token in doc[II]])
print([token.lex_id for token in doc[II]])
print([token.text for token in doc[II]])
print([nlp.vocab.strings[token.text] for token in doc[II]])

[15686, 33, 15686, 358821, 29900, 7, 3, 23757, 8, 15686, 18254, 62174, 13, 4085, 17115]
[15686, 33, 15686, 358821, 29900, 7, 3, 23757, 8, 15686, 18254, 62174, 13, 4085, 17115]
['electron', '-', 'electron', 'bremsstrahlung', 'emission', 'and', 'the', 'inference', 'of', 'electron', 'flux', 'spectra', 'in', 'solar', 'flares']
[14911849430818137050, 9153284864653046197, 14911849430818137050, 5074391312506727947, 7093072741639743635, 2283656566040971221, 7425985699627899538, 10770840912364104747, 886050111519832510, 14911849430818137050, 7386502683099369518, 721847661104588153, 3002984154512732771, 3825196732376443040, 54522159426843760]


In [19]:
# Words_vec= words in the text 
# Vocab = unique words in the text

import numpy as np

Words=[]
Max_Length=30;
Words_vec= np.empty([max_results,Max_Length],dtype='int')


for i in range(9):
  #len([token.lex_id for token in doc[i]])
  Words_vec[i,:len([token.lex_id for token in doc[i]])]=[token.lex_id for token in doc[i]]
  for token in doc[i]:
    #print(token.text, token.has_vector, token.vector, token.vector_norm ,"\n")
    #print(token.text, token.has_vector, token.lex_id ,"\n")
    Words.append(token.text.lower())
    

Vocab = []
[Vocab.append(x) for x in Words if x not in Vocab]
print('number of unique words in our data=',len(Vocab))

number of unique words in our data= 68


In [9]:
Input_list=[]
for sample in range(len(doc)):
  Input_list.append([token.rank for token in doc[sample]])
Output_list=Input_list;
Input_Output_Data_list=[Input_list,Output_list]

In [10]:
print(Input_list[6],doc[6])

[3371, 2322, 8, 429374] improved scenario of baryogenesis


In [11]:
## next: to understand / complete

train_list=  Input_list
label=Output_list
validation_list=train_list
test_list=train_list

In [12]:
import torch
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

def collate_batch(batch):
    label_list, text_list = [], []
    for _sample in batch:
        label_list.append(torch.tensor(_sample))
        text_list.append(torch.tensor(_sample))
    return pad_sequence(label_list, padding_value=0.0), pad_sequence(text_list, padding_value=0.0)

batch_size = 5

def create_iterators(batch_size=batch_size):
    """Heler function to create the iterators"""
    dataloaders = []
    for split in [train_list, validation_list, test_list]:
        dataloader = DataLoader(
            split, batch_size=batch_size,
            collate_fn=collate_batch
            )
        dataloaders.append(dataloader)
    return dataloaders

train_iterator, valid_iterator, test_iterator = create_iterators()

next(iter(train_iterator))




(tensor([[  1868,  15686,  15686,  15686, 247117],
         [     8,   9884,  14652,   3405,      8],
         [ 15686,  49985,     64, 198147,      6],
         [    33,  27361,     57,      7,    181],
         [ 15686,      5, 188891,  15686,     33],
         [ 34898,  23120,  15686,   8928,  15686],
         [    20,    331,   1908,  13132,    376],
         [  9055,  24230,      5,     57,     24],
         [  4167,  13604,      0,  15686,    469],
         [ 21514,      0,      0,  17913,     33],
         [     0,      0,      0,  98497,  15686],
         [     0,      0,      0,     13,      7],
         [     0,      0,      0,      3,  15686],
         [     0,      0,      0,   4085,     33],
         [     0,      0,      0,   2647,   1908],
         [     0,      0,      0,      0,   1180],
         [     0,      0,      0,      0,     13],
         [     0,      0,      0,      0,      6],
         [     0,      0,      0,      0,    193],
         [     0,      0,      